In [1]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import pandas as pd
from cloudy_fit_lib import *
from scipy.interpolate import RegularGridInterpolator
import pickle

In [2]:
plt.style.use('/Users/thepoetoftwilight/Documents/CUBS/Code/science.mplstyle')

In [3]:
rootdir = '/Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/cloudy_grids/sub_grid/'

Obtain the UVB grid from FG20

In [4]:
fg20_z_grid, fg20_wav_grid, fg20_J_nu_grid = read_uvb('', 'fg20_galaxy.ascii')

Fetch the SED at $z=1$

In [5]:
z_test = 1

In [6]:
fg20_J_nu_test = fetch_sed(z_test, fg20_z_grid, fg20_J_nu_grid)

Specify $N_{\mathrm{HI}}$ grid points and get corresponding filenames

In [7]:
logN_HI_min = 12
logN_HI_max = 17
logN_HI_step = 0.25

logN_HI_arr = np.arange(logN_HI_min, logN_HI_max+logN_HI_step, logN_HI_step)

In [8]:
file_list = create_grid_file_list(logN_HI_arr)

Specify grid points for densities and metallicities

In [9]:
log_hdens_min = -4.25
log_hdens_max = 1
log_hdens_step = 0.25

log_hdens_arr = np.arange(log_hdens_min, log_hdens_max+log_hdens_step, log_hdens_step)

In [10]:
log_metals_min = -0.25
log_metals_max = 1
log_metals_step = 0.25

log_metals_arr = np.arange(log_metals_min, log_metals_max+log_metals_step, log_metals_step)

In [11]:
# Total number of grid points
len(logN_HI_arr)*len(log_hdens_arr)*len(log_metals_arr)

2772

Process the grid to identify "failures"

In [12]:
log_T_pie_thresh = 4

In [13]:
# Bifurcate failures based on stopping temperature
low_temp_failures = []
high_temp_failures = []

In [14]:
for i in range(len(logN_HI_arr)):
    
    logN_HI = logN_HI_arr[i]
    
    # The filename corresponding to the current stopping HI column density
    filename = file_list[i]
    
    # Get list of densities and metallicities for this stopping HI column density 
    log_hdens_grid, log_metals_grid = read_grd_file(rootdir, filename)
    
    # Get average (log) HI temperatures for all grid points
    log_temps_grid = read_avr_file(rootdir, filename)
    
    # Get column densities for all species
    species_names, log_col_dens_grid = read_col_file(rootdir, filename)
    
    for j in range(len(log_hdens_arr)):
        
        log_hdens = log_hdens_arr[j]
        
        for k in range(len(log_metals_arr)):
            
            log_metals = log_metals_arr[k]
            
            # Get grid index number for the current n_H and metallicity
            idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]
            
            # Isolate the average temperature and column density for all species
            log_temp = log_temps_grid[idx]
            log_col_dens = log_col_dens_grid[idx]
            
            # Check if this grid point is a failure
            if np.round(log_col_dens[0],2) != logN_HI:
                # Categorize based on temperature
                if log_temp<log_T_pie_thresh:
                    low_temp_failures.append([logN_HI, log_hdens, log_metals, log_temp, log_col_dens[0]])
                else:
                    high_temp_failures.append([logN_HI, log_hdens, log_metals, log_temp, log_col_dens[0]])

/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(co

In [15]:
low_temp_failures = np.array(low_temp_failures)
high_temp_failures = np.array(high_temp_failures)

In [16]:
len(low_temp_failures)

0

In [17]:
len(high_temp_failures)

0

Save the sub grid

In [18]:
species_logN_samples = {}

In [23]:
for i in range(len(logN_HI_arr)):
    
    logN_HI = logN_HI_arr[i]
    
    # The filename corresponding to the current stopping HI column density
    filename = file_list[i]
    
    # Get list of densities and metallicities for this stopping HI column density 
    log_hdens_grid, log_metals_grid = read_grd_file(rootdir, filename)
    
    # Get average (log) HI temperatures for all grid points
    log_temps_grid = read_avr_file(rootdir, filename)
    
    # Get column densities for all species
    species_names, log_col_dens_grid = read_col_file(rootdir, filename)
    
    for j in range(len(log_hdens_arr)):
        
        log_hdens = log_hdens_arr[j]
        
        for k in range(len(log_metals_arr)):
            
            log_metals = log_metals_arr[k]
            
            # Get grid index number for the current n_H and metallicity
            idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]
            
            # Isolate the average temperature and column density for all species
            log_temp = log_temps_grid[idx]
            log_col_dens = log_col_dens_grid[idx]
            
            # For each species
            for l in range(len(species_names)):
                
                s = species_names[l]
                
                if s not in species_logN_samples.keys():
                    
                    species_logN_samples[s] = -99.*np.ones((len(logN_HI_arr), 
                                                            len(log_hdens_arr),
                                                            len(log_metals_arr)))
                
                # Check for photoionized gas and converged logN(HI)
                if np.round(log_col_dens[0],2) == logN_HI:
                    species_logN_samples[s][i,j,k] = log_col_dens[l]

/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/Users/thepoetoftwilight/Documents/CUBS/Code/cloudy_fitting/cloudy_fit_lib.py:206: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(co

In [26]:
np.where(logN_HI_arr==15)

(array([12]),)

In [27]:
np.where(log_metals_arr==1)

(array([5]),)

In [32]:
log_hdens_arr

array([-4.25, -4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25,
       -2.  , -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,
        0.25,  0.5 ,  0.75,  1.  ])

In [31]:
species_logN_samples['C+3'][12,:,5]

array([16.0102024 , 15.95285054, 15.82239563, 15.60548498, 15.30007132,
       14.91829723, 14.47606899, 13.98245306, 13.43898986, 12.84109969,
       12.17954892, 11.44776097, 10.65739244,  9.83690743,  9.01828431,
        8.23155878,  7.50073033,  6.83970174,  6.25104416,        -inf,
        3.56053548,  3.13534259])

In [21]:
# Save the main grid
output = open(rootdir+'sub_grid.pkl', 'wb')
pickle.dump(species_logN_samples, output)
output.close()